In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

%%time
%run ../py/sql_utlis.py

sqlu = SqlUtilities()
_, CURSOR = sqlu.get_jh_conn_cursor()

Wall time: 10.1 s


In [3]:

# Make a dictionary of all the HTML strings
sql_str = f'''
    SELECT
        np.[navigable_parent_id],
        np.[navigable_parent]
    FROM [Jobhunting].[dbo].[NavigableParents] np;'''
df = pd.DataFrame(sqlu.get_execution_results(CURSOR, sql_str, verbose=False))
html_str_dict = df.set_index('navigable_parent_id').navigable_parent.to_dict()

In [36]:

def guess_seq_len(seq, verbose=False):
    seq_len = 1
    initial_item = seq[0]
    butfirst_items = seq[1:]
    if initial_item in butfirst_items:
        first_match_idx = butfirst_items.index(initial_item) + 1
        if verbose:
            print(f'"{initial_item}" was found at index 0 and index {first_match_idx}')
        max_seq_len = min(len(seq) - first_match_idx, first_match_idx)
        for seq_len in range(max_seq_len, 0, -1):
            if seq[:seq_len] == seq[first_match_idx:first_match_idx+seq_len]:
                if verbose:
                    print(f'A sequence length of {seq_len} was found at index {first_match_idx}')
                break
    
    return seq_len

In [195]:

# Find the largest number of duplicates
sql_str = '''
    SELECT *
    FROM [Jobhunting].[dbo].[NavigableParentSequence];'''
nps_df = pd.DataFrame(sqlu.get_execution_results(CURSOR, sql_str, verbose=False))

In [196]:

# Get file with most-duplicated HTML string
columns_list = ['file_name_id', 'navigable_parent_id']
df = nps_df.groupby(columns_list, as_index=False).size().sort_values('size', ascending=False)
file_name_ids_list = df.file_name_id.tolist()
file_name_id = file_name_ids_list[1]

In [197]:

# Delete initial duplicate sequences
sql_str = f'''
    SELECT nps.[navigable_parent_id]
    FROM [Jobhunting].[dbo].[NavigableParentSequence] nps
    WHERE nps.[file_name_id] = {file_name_id}
    ORDER BY nps.sequence_order;'''
sequence_df = pd.DataFrame(sqlu.get_execution_results(CURSOR, sql_str, verbose=False))
navigable_parent_ids_list = sequence_df.navigable_parent_id.tolist()
cutoff = guess_seq_len(navigable_parent_ids_list)
while cutoff > 1:
    sql_str = f'''
        DELETE FROM [Jobhunting].[dbo].[NavigableParentSequence]
        WHERE
            [sequence_order] < {cutoff} AND
            [file_name_id] = {file_name_id};'''
    CURSOR.execute(sql_str)
    sql_str = f'''
        UPDATE [Jobhunting].[dbo].[NavigableParentSequence]
        SET [sequence_order] = [sequence_order] - {cutoff}
        WHERE [file_name_id] = {file_name_id};'''
    CURSOR.execute(sql_str)
    sql_str = f'''
        SELECT nps.[navigable_parent_id]
        FROM [Jobhunting].[dbo].[NavigableParentSequence] nps
        WHERE nps.[file_name_id] = {file_name_id}
        ORDER BY nps.sequence_order;'''
    sequence_df = pd.DataFrame(sqlu.get_execution_results(CURSOR, sql_str, verbose=False))
    navigable_parent_ids_list = sequence_df.navigable_parent_id.tolist()
    cutoff = guess_seq_len(navigable_parent_ids_list)

In [198]:

# Show the partially-cleaned sequence
sql_str = f'''
    SELECT *
    FROM [Jobhunting].[dbo].[NavigableParentSequence] nps
    WHERE nps.[file_name_id] = {file_name_id}
    ORDER BY nps.sequence_order;'''
sequence_df = pd.DataFrame(sqlu.get_execution_results(CURSOR, sql_str, verbose=False))
sequence_df

,navigable_parent_sequence_id,file_name_id,navigable_parent_id,sequence_order,mrs_id
0,5312,93,3104,0,0
1,5313,93,3119,1,0
2,5314,93,14264,2,0
3,5315,93,14261,3,0
4,5316,93,3119,4,0
...,...,...,...,...,...
141,5454,93,2641,142,0
142,5455,93,15419,143,0
143,5456,93,2541,144,0
144,5457,93,2641,145,0



## Clean up the sequence order numbers

In [215]:

def find_largest_repeating_sequence(sequence_df):
    '''Find the first index of the largest repeating sequence'''
    max_start_idx = -1
    max_cutoff = -1
    navigable_parent_ids_list = sequence_df.navigable_parent_id.tolist()
    for start_idx in range(len(navigable_parent_ids_list)):
        cutoff = guess_seq_len(navigable_parent_ids_list[start_idx:])
        if max_cutoff < cutoff:
            max_cutoff = cutoff
            max_start_idx = start_idx
    
    return max_start_idx, max_cutoff

In [216]:

# Get the file IDs
sql_str = f'''
    SELECT fn.[file_name_id]
    FROM [Jobhunting].[dbo].[FileNames] fn;'''
file_name_ids_list = pd.DataFrame(sqlu.get_execution_results(CURSOR, sql_str,
                                                           verbose=False)).file_name_id.tolist()
for file_name_id in file_name_ids_list:
    sql_str = f'''
        SELECT *
        FROM [Jobhunting].[dbo].[NavigableParentSequence] nps
        WHERE nps.[file_name_id] = {file_name_id}
        ORDER BY nps.sequence_order;'''
    sequence_df = pd.DataFrame(sqlu.get_execution_results(CURSOR, sql_str, verbose=False))
        
    # Find the first index of the largest repeating sequence
    max_start_idx, max_cutoff = find_largest_repeating_sequence(sequence_df)
    
    while max_cutoff > 1:
            
        # Delete the first occurance of that sequence
        sequence_ids_list = sequence_df.navigable_parent_sequence_id.tolist()
        ids_list = [str(id) for id in sequence_ids_list[max_start_idx:max_start_idx+max_cutoff]]
        ids_list_str = "('" + "', '".join(ids_list) + "')"
        sql_str = f'''
            DELETE FROM [Jobhunting].[dbo].[NavigableParentSequence]
            WHERE [navigable_parent_sequence_id] IN {ids_list_str};'''
        CURSOR.execute(sql_str)
        CURSOR.commit()

        # Reset the sequence order numbers
        sql_str = f'''
            SELECT *
            FROM [Jobhunting].[dbo].[NavigableParentSequence] nps
            WHERE nps.[file_name_id] = {file_name_id}
            ORDER BY nps.sequence_order;'''
        sequence_df = pd.DataFrame(sqlu.get_execution_results(CURSOR, sql_str, verbose=False))
        sequence_ids_list = sequence_df.navigable_parent_sequence_id.tolist()
        count = 0
        for nps_id in sequence_ids_list:
            sql_str = f'''
                UPDATE [Jobhunting].[dbo].[NavigableParentSequence]
                SET sequence_order = {count}
                WHERE (navigable_parent_sequence_id = {nps_id});'''
            CURSOR.execute(sql_str)
            count += 1
        CURSOR.commit()
        
        # Find the first index of the largest repeating sequence
        sql_str = f'''
            SELECT *
            FROM [Jobhunting].[dbo].[NavigableParentSequence] nps
            WHERE nps.[file_name_id] = {file_name_id}
            ORDER BY nps.sequence_order;'''
        sequence_df = pd.DataFrame(sqlu.get_execution_results(CURSOR, sql_str, verbose=False))
        max_start_idx, max_cutoff = find_largest_repeating_sequence(sequence_df)


## Visualize the Cleanup

In [199]:

# Get the file name
sql_str = f'''
    SELECT fn.[file_name]
    FROM [Jobhunting].[dbo].[FileNames] fn
    WHERE fn.[file_name_id] = {file_name_id};'''
file_name = pd.DataFrame(sqlu.get_execution_results(CURSOR, sql_str, verbose=False)).file_name.squeeze().strip()

In [211]:

# Prep the temp file
file_path = os.path.abspath(os.path.join('../saves/html', file_name))
temp_file_path = os.path.abspath(os.path.join('../saves/html', f'temp_{file_name}'))
with open(temp_file_path, 'w') as f:
    f.write('')

In [212]:

# Display the HTML for our cleaned-up sequence
# and prepare it for comparison to the original file
from IPython.display import HTML, display

sql_str = f'''
    SELECT *
    FROM [Jobhunting].[dbo].[NavigableParentSequence] nps
    WHERE nps.[file_name_id] = {file_name_id}
    ORDER BY nps.sequence_order;'''
sequence_df = pd.DataFrame(sqlu.get_execution_results(CURSOR, sql_str, verbose=False))
navigable_parent_ids_list = sequence_df.navigable_parent_id.tolist()
with open(temp_file_path, 'a') as f:
    for navigable_parent_id in navigable_parent_ids_list:
        display(HTML(html_str_dict[navigable_parent_id]))
        f.write(html_str_dict[navigable_parent_id] + '\n')

In [118]:

# Running Compare It! to compare the old and new HTML files
compareit_str = 'C:\\Program Files (x86)\\Compare It!\\wincmp3.exe'
!"{compareit_str}" "{file_path}" "{temp_file_path}"


----

In [11]:

# View the duplicates
mask_series = sequence_df.duplicated(subset='navigable_parent_id')
sequence_df[mask_series]

,navigable_parent_sequence_id,file_name_id,navigable_parent_id,sequence_order,mrs_id
21,9496,175,15950,21,0
22,9497,175,1842,22,0
23,9498,175,15546,23,0
27,9502,175,1305,27,0
29,9504,175,1842,29,0
...,...,...,...,...,...
210,9685,175,2634,210,0
211,9686,175,15419,211,0
212,9687,175,2512,212,0
213,9688,175,2634,213,0


In [ ]:

# Delete consecutively-duplicated HTML strings
sql_str = '''
    DELETE FROM [Jobhunting].[dbo].[NavigableParentSequence]
    WHERE [navigable_parent_sequence_id] IN (
        SELECT h1.[navigable_parent_sequence_id]
        FROM
            [Jobhunting].[dbo].[NavigableParentSequence] AS h1 INNER JOIN
            [Jobhunting].[dbo].[NavigableParentSequence] AS h2 ON
                h1.[file_name_id] = h2.[file_name_id] AND
                h2.[navigable_parent_id] = h1.[navigable_parent_id] AND
                h2.[sequence_order] = h1.[sequence_order] - 1
        );'''
CURSOR.execute(sql_str)
CURSOR.commit()

In [13]:

# Get a list of html strings that repeat somewhere in the posting
sql_str = f'''
    SELECT *
    FROM [Jobhunting].[dbo].[NavigableParentSequence] nps
    WHERE nps.[file_name_id] = {file_name_id}
    ORDER BY nps.sequence_order;'''
sequence_df = pd.DataFrame(sqlu.get_execution_results(CURSOR, sql_str, verbose=False))
navigable_parent_ids_list = sequence_df.navigable_parent_id.unique().tolist()
for navigable_parent_id in navigable_parent_ids_list:
    
    # Get their sequence numbers
    mask_series = (sequence_df.navigable_parent_id == navigable_parent_id)
    sequence_orders_list = sequence_df[mask_series].sequence_order.tolist()
    
    # Get the next html strings
    next_tags_list = []
    for sequence_order in sequence_orders_list:
        mask_series = (sequence_df.sequence_order == sequence_order+1)
        next_tag = sequence_df[mask_series].navigable_parent_id.squeeze()
        if type(next_tag) != pd.Series:
            next_tags_list.append(next_tag)
    
    # Check for sequence duplication
    if len(next_tags_list) != len(set(next_tags_list)):
        print(navigable_parent_id, next_tags_list)

3117 [2564, 2564, 2564, 2564]
2564 [14232, 14232, 14232, 14232, 1443]
14232 [3116, 3117, 3117, 2564]
1443 [3058, 3058, 3058, 3058, 3058]
3058 [14495, 14495, 14495, 14495, 14495]
14495 [15038, 15038, 15038, 15038, 15038]
15038 [3085, 3085, 3085, 3085, 3085]
3085 [1423, 1423, 1423, 1443, 1423]
1423 [1305, 1305, 1305, 1305, 1305]
1305 [15049, 15547, 15049, 15547, 15049, 15547, 15049, 15049, 15049, 15547, 15547, 15547]
15049 [1931, 1931, 1931, 1931, 1931, 1931]
1931 [15950, 15950, 15950, 15950, 15950, 15950]
15950 [1842, 1842, 1842, 1842, 1842, 1842, 1842, 1842, 1842, 1842, 1842, 1842]
1842 [15546, 15546, 15546, 15546, 15546, 15546, 15546, 15546, 15546, 15546, 15546, 15546, 15546, 15546, 15546, 15546, 15546, 15546]
15546 [1424, 1422, 444, 1424, 1422, 444, 1424, 1422, 444, 1423, 1305, 1424, 1424, 753, 1422, 1422, 689, 444]
1424 [753, 753, 753, 753, 753]
753 [14771, 14771, 14771, 14771, 14771, 14771]
14771 [15559, 15559, 15559, 15559, 15559, 15559]
15559 [2434, 2434, 2434, 2434, 2434, 2434]


In [ ]:

# Find the largest job postings
sql_str = '''
    SELECT
        nps.[file_name_id],
        COUNT(nps.[file_name_id]) AS total_ids
    FROM [Jobhunting].[dbo].[NavigableParentSequence] nps
    GROUP BY [file_name_id]
    ORDER BY COUNT(nps.[file_name_id]) DESC;'''
counts_df = pd.DataFrame(sqlu.get_execution_results(CURSOR, sql_str, verbose=False))
counts_df

In [ ]:

file_name_id = counts_df.head(1).file_name_id.squeeze()
sql_str = f'''
    SELECT
        fn.[file_name],
        np.[navigable_parent_id],
        nps.[sequence_order]
    FROM
        [Jobhunting].[dbo].[FileNames] fn INNER JOIN
        [Jobhunting].[dbo].[NavigableParentSequence] nps ON
        fn.[file_name_id] = nps.[file_name_id] INNER JOIN
        [Jobhunting].[dbo].[NavigableParents] np ON
        nps.[navigable_parent_id] = np.[navigable_parent_id]
    WHERE fn.[file_name_id] = {file_name_id}
    ORDER BY nps.sequence_order;'''
sequence_df = pd.DataFrame(sqlu.get_execution_results(CURSOR, sql_str, verbose=False))
sequence_df